# Analysing Montecarlo Packets

`RPacketPlotter` plots the `RPackets` that are generated by the [Montecarlo](https://tardis-sn.github.io/tardis/physics/montecarlo/index.html) method and creates an animated plot that contains the packet trajectories as they move away from the photosphere.
The properties of individual RPackets are taken from the [rpacket_tracker](https://tardis-sn.github.io/tardis/io/output/rpacket_tracking.html).

`RPacketPlotter` uses the properties (specifically, `mu` and `r`) present in the `rpacket_tracker` to calculate the coordinates of packets as they move through the ejecta. In the following section, the mathematical expression for getting the angle(θ) of packets with respect to the x-axis is shown, which can be used (along with radius `r`) to calculate the x and y coordinates of packets.

## Getting packet coordinates

`RPacketPlotter` uses the properties (specifically, `mu` and `r`) present in the `rpacket_tracker` to calculate the coordinates of packets as they move through the ejecta. In the following section, the mathematical expression for getting the angle(θ) of packets with respect to the x-axis is shown, which can be used (along with radius `r`) to calculate the x and y coordinates of packets.
<br><br>
<img src="attachment:packet_diagram.jpg" style="width:400px">

<br>The diagram above shows the packet trajectory as it starts from photosphere `P0` and continues to move along the subsequent points `P1`, `P2`, and so on.

<div class="alert alert-info">

Note
    
Here `μ` represents the direction of packet propagation with respect to the radial line.
    
</div>

To determine the polar coordinates of any arbitrary point, say `P2`, we need `r2` and `θ2`. `r2` is already present in the array obtained from the simulation. To determine `θ2`, we use the sine rule and apply it to the triangle `OP1P2`, where `O` is the center.

$$
\frac{r_{2}}{\sin(\pi - \mu_{1})} = \frac{r_{1}}{\sin(\alpha)}
$$

Now, writing `α` in terms of `μ1` and `θ2`

$$ 
α = μ_{1} - θ_{2}
$$
$$
\frac{r_{2}}{\sin(\pi - \mu_{1})} = \frac{r_{1}}{\sin(μ_{1} - θ_{2})}
$$

Thus,

$$ 
θ_{2} = -\sin^{-1}(\frac{r1}{r2}\sin(\mu_{1})) + \mu_{1}
$$

Hence, for `i-th` point, `θ` will be:

$$ 
θ_{i} = -\sin^{-1}(\frac{r_{i-1}}{r_{i}}\sin(\mu_{i-1})) + \mu_{i-1}
$$

## Running the simulation

In [46]:
import math

import astropy.units as u
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from tardis import run_tardis
from tardis.io.atom_data import download_atom_data
from tardis.io.configuration.config_reader import Configuration

# We download the atomic data needed to run the simulation
download_atom_data('kurucz_cd23_chianti_H_He')

In [47]:
# Reading the Configuration stored in `tardis_example.yml` into config

config = Configuration.from_yaml("tardis_example.yml")

In [48]:
# changing config file for enabling the rpacket_tracking

config["montecarlo"]["tracking"]["track_rpacket"]=True

In [49]:
sim = run_tardis(config, show_progress_bars=False, log_level='ERROR')

## Plotting Packets with RPacketPlotter

Importing the RPacketPlotter

In [50]:
from tardis.visualization import RPacketPlotter

Now, we create an RPacketPlotter object `rpacket_plotter` that will be used to generate a plot.

`no_of_packets` can be specified as a parameter to the `from_simulation` class method. By default, `15` packets will used to create a plot.

In [51]:
rpacket_plotter = RPacketPlotter.from_simulation(sim, no_of_packets=20)

Using the `rpacket_plotter` we use the `generate_plot` method to create a plot.

Here the `theme` parameter can be defined. Currently, we have 2 themes, i.e. `light` and `dark`. By Default the `light` theme will be plotted.

In [52]:
# rpacket_plotter.generate_plot().show(renderer="notebook_connected")

## Calculating Data

In [53]:
theme_colors = dict(
            light=dict(
                linecolor="#555",
                gridcolor="#fafafa",
                zerolinecolor="#fafafa",
                color="#000",
                photosphere_line_color="black",
                photosphere_fillcolor="darkgrey",
                shells_line_color="black",
                packet_line_color="darkslategrey",
                plot_bgcolor="#fafafa",
                paper_bgcolor="#fafafa",
                title_font_color="#444",
                legendgrouptitle_color="#444",
                button_bgcolor="#fafafa",
                button_font_color="#2A3F5F",
                slider_font_color="#2A3F5F",
                bordercolor="#BEC8D9",
                slider_bgcolor="#F8FAFC",
                slider_activebgcolor="#DBDDE0",
                slider_currentvalue_color="#2A3F5F",
                font_color="#000",
            ))

interaction_from_num = [
            {"text": "No Interaction", "color": "darkslategrey", "opacity": 0},
            {"text": "e-Scattering", "color": "#3366FF", "opacity": 1},
            {"text": "Line Interaction", "color": "#FF3300", "opacity": 1},
        ]

In [54]:
no_of_packets=20
theme="light"

# getting velocity of different shells
v_shells = sim.simulation_state.velocity.to_value(u.km / u.s)
r_packet_tracker = sim.transport.transport_state.rpacket_tracker_df.loc[
        0 : (no_of_packets)
    ]



In [55]:
# for plotting packets at equal intervals throught the circle, we choose thetas distributed uniformly
thetas = np.linspace(0, 2 * math.pi, no_of_packets + 1)
rpackets_x = []
rpackets_y = []
rpackets_interactions = []
# getting coordinates and interaction arrays for all packets
for packet_no in range(no_of_packets):
    r_track =  r_packet_tracker.loc[packet_no]["r"]
    mu_track = r_packet_tracker.loc[packet_no]["mu"]
    time = sim.simulation_state.time_explosion.value
    last_interaction_type = r_packet_tracker.loc[packet_no]["interaction_type"]
    theta_initial = thetas[packet_no]

    single_rpacket_x, single_rpacket_y, single_theta, single_rpacket_interactions = [], [], [], []

        # getting thetas at different steps of the packet movement

    for step_no in range(len(r_track)):
        # for the first step the packet is at photosphere, so theta will be equal to the intial angle we are launching the packet from
        if step_no == 0:
            single_theta.append(theta_initial)
        # for further steps we calculate thetas with the formula derived in the documentation
        else:
            if r_track[step_no] < r_track[step_no - 1]:
                single_theta.append(
                    single_theta[-1]
                    - math.pi
                    + math.asin(
                        r_track[step_no - 1]
                        * math.sin(math.acos(mu_track[step_no - 1]))
                        / r_track[step_no]
                    )
                    + math.acos(mu_track[step_no - 1])
                )
            else:
                single_theta.append(
                    single_theta[-1]
                    + math.asin(
                        -1
                        * r_track[step_no - 1]
                        * math.sin(math.acos(mu_track[step_no - 1]))
                        / r_track[step_no]
                    )
                    + math.acos(mu_track[step_no - 1])
                )

    # converting the thetas into x and y coordinates using radius as radius*cos(theta) and radius*sin(theta) respectively
    single_rpacket_x = (np.array(r_track)) * np.cos(np.array(single_theta)) * 1e-5 / time
    single_rpacket_y = (np.array(r_track)) * np.sin(np.array(single_theta)) * 1e-5 / time

    # adding interactions at different steps
    # using the change of slope of the trajectory line at different steps, we determine if an interactions happened or not.

    for step_no in range(len(r_track)):
        # when packet is at its starting and ending point in its trajectory, we consider it as no interaction
        if step_no == 0 or step_no == len(r_track) - 1:
            single_rpacket_interactions.append(0)
        else:
            # current slope is the slope of line from previous position of the packet to the current position
            single_rpacket_interactions.append(last_interaction_type[step_no])

    rpackets_x.append(single_rpacket_x)
    rpackets_y.append(single_rpacket_y)
    rpackets_interactions.append(single_rpacket_interactions)

np_rpackets_x = np.array(rpackets_x, dtype="object")
np_rpackets_y = np.array(rpackets_y, dtype="object")
np_rpackets_interactions = np.array(rpackets_interactions, dtype="object")


In [58]:
rpacket_step_no_array_max_size = max(list(map(len, np_rpackets_x)))

for packet_no in range(len(np_rpackets_x)):
    # making all coordinate arrays of size `rpacket_step_no_array_max_size` by repeating the last element across the remaining length of array
    np_rpackets_x[packet_no] = np.append(
        np_rpackets_x[packet_no],
        np_rpackets_x[packet_no][-1]
        * np.ones(
            [rpacket_step_no_array_max_size - len(np_rpackets_x[packet_no])]
        ),
    )
    np_rpackets_y[packet_no] = np.append(
        np_rpackets_y[packet_no],
        np_rpackets_y[packet_no][-1]
        * np.ones(
            [rpacket_step_no_array_max_size - len(np_rpackets_y[packet_no])]
        ),
    )
    np_rpackets_interactions[packet_no] = np.append(
        np_rpackets_interactions[packet_no],
        np_rpackets_interactions[packet_no][-1]
        * np.ones(
            [
                rpacket_step_no_array_max_size
                - len(np_rpackets_interactions[packet_no])
            ]
        ),
    )

In [ ]:
fig = go.Figure()
# Set axes properties
fig.update_xaxes(
    scaleanchor="y",
    scaleratio=1,
    range=[-1.1 * v_shells[-1], 1.1 * v_shells[-1]],
    title="Velocity (km/s)",
    exponentformat="none",
    color=theme_colors[theme]["color"],
    linecolor=theme_colors[theme]["linecolor"],
    gridcolor=theme_colors[theme]["gridcolor"],
    zerolinecolor=theme_colors[theme]["zerolinecolor"],
)
fig.update_yaxes(
    range=[-1.1 * v_shells[-1], 1.1 * v_shells[-1]],
    title="Velocity (km/s)",
    exponentformat="none",
    color=theme_colors[theme]["color"],
    linecolor=theme_colors[theme]["linecolor"],
    gridcolor=theme_colors[theme]["gridcolor"],
    zerolinecolor=theme_colors[theme]["zerolinecolor"],
)

# adding the shells and photosphere
for shell_no in range(len(sim.simulation_state.radius.value)):
    if shell_no == 0:
        # photosphere
        fig.add_shape(
            type="circle",
            xref="x",
            yref="y",
            x0=-1 * v_shells[shell_no],
            y0=-1 * v_shells[shell_no],
            x1=v_shells[shell_no],
            y1=v_shells[shell_no],
            line_color=theme_colors[theme][
                "photosphere_line_color"
            ],
            fillcolor=theme_colors[theme]["photosphere_fillcolor"],
            opacity=1,
        )
    elif shell_no == (len(sim.simulation_state.radius.value) - 1):
        # outermost shell
        fig.add_shape(
            type="circle",
            xref="x",
            yref="y",
            x0=-1 * v_shells[shell_no],
            y0=-1 * v_shells[shell_no],
            x1=v_shells[shell_no],
            y1=v_shells[shell_no],
            line_color=theme_colors[theme]["shells_line_color"],
            opacity=1,
        )
    else:
        # remaining shells
        fig.add_shape(
            type="circle",
            xref="x",
            yref="y",
            x0=-1 * v_shells[shell_no],
            y0=-1 * v_shells[shell_no],
            x1=v_shells[shell_no],
            y1=v_shells[shell_no],
            line_color=theme_colors[theme]["shells_line_color"],
            opacity=0.2,
        )


for packet_no in range(len(np_rpackets_x)):
    fig.add_trace(
        go.Scatter(
            x=np_rpackets_x[packet_no],
            y=np_rpackets_y[packet_no],
            mode="markers+lines",
            name="Packet " + str(packet_no + 1),
            showlegend=False,
            hovertemplate="<b>X</b>: %{x}"
            + "<br><b>Y</b>: %{y}<br>"
            + "<b>Last Interaction: %{text}</b>",
            text=[
                interaction_from_num[
                    int(np_rpackets_interactions[packet_no][step_no])
                ]["text"]
                for step_no in range(len(np_rpackets_x[packet_no]))
            ],
            line=dict(
                color=theme_colors[theme]["packet_line_color"]
            ),
            marker=dict(
                opacity=[
                    interaction_from_num[
                        int(np_rpackets_interactions[packet_no][step_no])
                    ]["opacity"]
                    for step_no in range(len(np_rpackets_x[packet_no]))
                ],
                color=[
                    interaction_from_num[
                        int(np_rpackets_interactions[packet_no][step_no])
                    ]["color"]
                    for step_no in range(len(np_rpackets_x[packet_no]))
                ],
            ),
        )
    )

# adding legends
fig.add_trace(
    go.Scatter(
        x=[9999999],
        y=[0],
        legendgroup="a",
        opacity=1,
        legendgrouptitle=dict(
            font=dict(
                color=theme_colors[theme]["legendgrouptitle_color"]
            ),
            text="Interaction Type:",
        ),
        mode="lines+markers",
        name="e-scattering",
        hoverlabel=dict(font=dict(color="#222")),
        marker=dict(color="#3366FF"),
    )
)
fig.add_trace(
    go.Scatter(
        x=[9999999],
        y=[0],
        legendgroup="a",
        opacity=1,
        mode="lines+markers",
        name="Line Interaction",
        marker=dict(color="#FF3300"),
    )
)

# Set figure size
fig.layout.plot_bgcolor = theme_colors[theme]["plot_bgcolor"]
fig.layout.paper_bgcolor = theme_colors[theme][
    "paper_bgcolor"
]

fig.update_layout(
    width=820,
    height=680,
    title="Packet Trajectories",
    title_font_color=theme_colors[theme]["title_font_color"],
    font_color=theme_colors[theme]["font_color"],
    updatemenus=[
        dict(
            type="buttons",
            xanchor="right",
            x=0.1,
            y=0,
            yanchor="top",
            direction="left",
            pad={"r": 10, "t": 87},
            showactive=False,
            bgcolor=theme_colors[theme]["button_bgcolor"],
            bordercolor=theme_colors[theme]["bordercolor"],
            font={
                "color": theme_colors[theme]["button_font_color"]
            },
            buttons=[
                {
                    "args": [
                        None,
                        {
                            "frame": {"duration": 500, "redraw": False},
                            "fromcurrent": True,
                            "transition": {
                                "duration": 300,
                                "easing": "quadratic-in-out",
                            },
                        },
                    ],
                    "label": "Play",
                    "method": "animate",
                },
                {
                    "args": [
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": "Pause",
                    "method": "animate",
                },
            ],
        )
    ],
)

# adding frames
for frame in range(rpacket_step_no_array_max_size + 1):
    frame_data = []
    for packet_no in range(len(np_rpackets_x)):
        # adding a scatter object containing the trajectory of a packet upto a particular frame number
        frame_data.append(
            go.Scatter(
                x=np_rpackets_x[packet_no].tolist()[0:frame],
                y=np_rpackets_y[packet_no].tolist()[0:frame],
                mode="markers+lines",
                name="Packet " + str(packet_no + 1),
                showlegend=False,
                hovertemplate="<b>X</b>: %{x}"
                + "<br><b>Y</b>: %{y}<br>"
                + "<b>Last Interaction: %{text}</b>",
                text=[
                    interaction_from_num[
                        int(np_rpackets_interactions[packet_no][step_no])
                    ]["text"]
                    for step_no in range(len(np_rpackets_x[packet_no]))
                ],
                line=dict(
                    color=theme_colors[theme]["packet_line_color"]
                ),
                marker=dict(
                    opacity=[
                        interaction_from_num[
                            int(np_rpackets_interactions[packet_no][step_no])
                        ]["opacity"]
                        for step_no in range(len(np_rpackets_x[packet_no]))
                    ],
                    color=[
                        interaction_from_num[
                            int(np_rpackets_interactions[packet_no][step_no])
                        ]["color"]
                        for step_no in range(len(np_rpackets_x[packet_no]))
                    ],
                ),
            )
        )
    fig.frames = [go.Frame(data=frame_data, name=frame)]

# adding timeline slider
slider_steps = []
for step_no in range(rpacket_step_no_array_max_size):
    slider_steps.append(
        {
            "args": [
                [step_no],
                {
                    "frame": {"duration": 300, "redraw": False},
                    "mode": "immediate",
                    "transition": {"duration": 300},
                },
            ],
            "label": step_no,
            "method": "animate",
        }
    )

fig.layout.sliders = [
    {
        "active": 0,
        "activebgcolor": theme_colors[theme][
            "slider_activebgcolor"
        ],
        "bgcolor": theme_colors[theme]["slider_bgcolor"],
        "bordercolor": theme_colors[theme]["bordercolor"],
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {
                "color": theme_colors[theme][
                    "slider_currentvalue_color"
                ],
            },
            "prefix": "Step:",
            "visible": True,
            "xanchor": "right",
        },
        "font": {
            "color": theme_colors[theme]["slider_font_color"]
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": slider_steps,
    }
]

fig.show(renderer="notebook_connected")